# Metrics of classification probabilities

TODO: change double float probs to integers, for less space/volume of data, could precompute from binned probabilities, check that it can be done in sql

also check the probabilities sum to 1 at some point
and allow submissions for negative of class
and flags for did not classify and reasons

In [1]:
%matplotlib inline
import sys
import os
import requests
import datetime
import dateutil.parser
import json
import numpy
import pandas
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics

In [2]:
 os.getenv("HOME")

'/global/homes/r/raknop'

In [4]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_"+username+"_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [5]:
rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [6]:
def run_query(query, subdict=None):
    if subdict == None:
        subdict = {}
    result = rqs.post( f'{url}/db/runsqlquery/',
        json={ 'query': query, 'subdict': subdict } )
    if result.status_code != 200:
        sys.stderr.write( f"ERROR: got status code {result.status_code} ({result.reason})\n" )
    else:
        data = json.loads( result.text )
        if ( 'status' not in data ) or ( data['status'] != 'ok' ):
            sys.stderr.write( "Got unexpected response\n" )
            print(data['error'])
        else:
            return data['rows']

In [7]:
rows = run_query( 'SELECT "classifierId","brokerName","brokerVersion",'
                  '"classifierName","classifierParams" '
                  'FROM elasticc_brokerclassifier' )
classifier_info = {}
for row in rows:
    classifier_info[row["classifierId"]] = row             
classifier_info

{36: {'classifierId': 36,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuess',
  'classifierParams': 'XGBUnified+Parsnip04'},
 37: {'classifierId': 37,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLive',
  'classifierParams': 'XGBUnified+Parsnip04'},
 38: {'classifierId': 38,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuessParsnip',
  'classifierParams': 'XGBUnified+Parsnip04'},
 39: {'classifierId': 39,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuessParsnipPrior',
  'classifierParams': 'XGBUnified+Parsnip04'},
 40: {'classifierId': 40,
  'brokerName': 'ALeRCE',
  'brokerVersion': '3.1.0',
  'classifierName': 'balto',
  'classifierParams': '3.1.0'},
 41: {'classifierId': 41,
  'brokerName': 'ALeRCE',
  'brokerVersion': '1.0.0',
  'classifierName': 'messi',
  'classifierParams': '1.0.0'},
 42: {'classifierId': 42,
  'broke

TODO: select just a couple classifiers/brokers

In [8]:
use_cfers = {i: classifier_info[i] for i in [40,44]}
use_cfers

{40: {'classifierId': 40,
  'brokerName': 'ALeRCE',
  'brokerVersion': '3.1.0',
  'classifierName': 'balto',
  'classifierParams': '3.1.0'},
 44: {'classifierId': 44,
  'brokerName': 'ANTARES',
  'brokerVersion': '2.0.0',
  'classifierName': 'LiCuParsnip_filter',
  'classifierParams': 'v2'}}

In [9]:
rows = run_query( 'SELECT DISTINCT ON ("classId") "classId",description '
                  'FROM elasticc_gentypeofclassid '
                  'ORDER BY "classId"' )
classname = {}
for row in rows:
    classname[row["classId"]] = row["description"]

In [10]:
classname

{0: 'Static/Other',
 1: 'Non-Recurring',
 2: 'Recurring',
 10: 'Non-Recurring/Other',
 11: 'SN-like',
 12: 'Fast',
 13: 'Long',
 20: 'Recurring/Other',
 21: 'Periodic',
 22: 'Non-Periodic',
 110: 'SN-like/Other',
 111: 'Ia',
 112: 'Ib/c',
 113: 'II',
 114: 'Iax',
 115: '91bg',
 120: 'Fast/Other',
 121: 'KN',
 122: 'M-dwarf Flare',
 123: 'Dwarf Nova',
 124: 'uLens',
 130: 'Long/Other',
 131: 'SLSN',
 132: 'TDE',
 133: 'ILOT',
 134: 'CART',
 135: 'PISN',
 210: 'Periodic/Other',
 211: 'Cepheid',
 212: 'RR Lyrae',
 213: 'Delta Scuti',
 214: 'EB',
 215: 'LPV/Mira',
 220: 'Non-Periodic/Other',
 221: 'AGN'}

## Everything above here is setup.  Run all that.  Then pick and choose below.

In [ ]:
# consider descIngestTimestamp, alertSentTimestamp
#
# alertSentTimestamp -- this gets updated in the database when the original
#    alert is sent out.  It's NOT present in the dev database, but should
#    be in the real one
# descIngestTimestamp -- just when we happend to suck in the broker message
#
# alertId -- opaque fingerprint, don't sort on it, but it does correspond to
#   the original alert.  Different alertId = different source, i.e.
#   measurement of the transient.

sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  "classifierId","diaObjectId","classId","alertId","trueClassId","probability","descIngestTimestamp" '
          'FROM elasticc_view_sourceclassifications '
          'ORDER BY "diaObjectId","descIngestTimestamp","classifierId" '
          'OFFSET 100 LIMIT 100' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data
# classifiers = data.classifierId.unique()

TODO: change double float probs to integers, for less space/volume of data
also check the probabilities sum to 1 at some point
and allow submissions for negative of class
and flags for did not classify and reasons

In [ ]:
classname

In [ ]:
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  "classifierId","diaObjectId","classId","alertId","trueClassId","probability","descIngestTimestamp","alertSentTimestamp" '
          'FROM elasticc_view_sourceclassifications '
          'WHERE "classifierId"=41 AND "classId"=111 AND "trueClassId"=111 '
          'ORDER BY "diaObjectId","descIngestTimestamp" '
          'OFFSET 100 LIMIT 100' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
import numpy as np
np.unique(data['diaObjectId'], return_counts=True)

In [ ]:
chosen = 1006052#1003722

In [ ]:
data[ data['diaObjectId'] == chosen]#1025381 ]

In [ ]:
# All these joins suggest we need a new view.
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  v."classifierId",v."diaObjectId",v."classId",v."alertId",v."trueClassId",v."probability",v."alertSentTimestamp", '
          '  s."midPointTai" '
          'FROM elasticc_view_sourceclassifications v '
          'INNER JOIN elasticc_diasource s ON v."diaSourceId"=s."diaSourceId" '
          'WHERE v."diaObjectId"='+str(chosen)+' AND "classId"=111 AND "trueClassId"=111 '
          'ORDER BY "classifierId","alertSentTimestamp" ' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
pandas.set_option('display.max_rows', 500)
data['t'] = data['alertSentTimestamp'].apply( lambda thing : dateutil.parser.isoparse( thing ) )
data.columns


In [ ]:
classifier_info

In [ ]:
for i,cfer in enumerate(classifiers):
    print(cfer,classifier_info[cfer]["brokerName"],classifier_info[cfer]["classifierName"],classifier_info[cfer]["brokerVersion"])

In [ ]:
blind = False

In [ ]:
    
# TODO: color for broker, linestyle for classifier

fig = plt.figure( figsize=(12,8), tight_layout=True )
ax = fig.add_subplot( 1,1,1)
classifiers = data['classifierId'].unique()
for i,cfer in enumerate(use_cfers):
    subdata = data[ data['classifierId'] == cfer ]
    label = f'{use_cfers[cfer]["brokerName"]} {use_cfers[cfer]["classifierName"]}'
    ax.plot( subdata['midPointTai'], subdata['probability'], color=f'C{i}', label=label )
    ax.tick_params( "both", labelsize=16 )
if ~blind:
    ax.legend()
    ax.set_title('diaObjectID='+str(chosen))#"Jackson Pollock" )
ax.set_xlabel('midPointTai (time)', fontsize=16)
ax.set_ylabel('probability of true class', fontsize=16)
fig.show()

1. metric as f(time relative to characteristic time)
2. metric as f(number of alerts)



In [ ]:
# Oh, the humanity
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  v."classifierId",v."diaObjectId",v."classId",v."alertId",v."probability",v."alertSentTimestamp", '
          '  s."midPointTai",t.peakmjd '
          'FROM elasticc_view_sourceclassifications v '
          'INNER JOIN elasticc_diasource s ON v."diaSourceId"=s."diaSourceId" '
          'INNER JOIN elasticc_diaobjecttruth t ON v."diaObjectId"=t."diaObjectId" '
          'WHERE v."diaObjectId" IN '
          '  ( SELECT "diaObjectId" FROM elasticc_diaobjecttruth WHERE gentype IN '
          '     ( SELECT gentype FROM elasticc_gentypeofclassid WHERE "classId"=111 ) '
          '    LIMIT 100 ) '
          'AND "classId"=111 '
          'ORDER BY "classifierId","alertSentTimestamp" ' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
data['Δt'] = data['midPointTai'] - data['peakmjd']
data

In [ ]:
onebroker = data[ data['classifierId'] == 40 ].copy()

In [ ]:
onebroker

In [ ]:
plt.hist( onebroker['Δt'] )

In [ ]:
# This is not the right way to bin.  We have to think harder about this.
# If there are more observations of some objectds, those objects will
# get overweighted.
onebroker['tbin'] = numpy.floor( ( onebroker['Δt'] + 20 ) / 5. )

In [ ]:
thing = onebroker.groupby( 'tbin' )['probability'].apply( 'mean' )

In [ ]:
plt.plot(thing)

In [14]:
tuple(use_cfers.keys())

(40, 44)

## New Query, Alex and Rob, 2022-12-15, Waiting For A Better Title

In [22]:
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  v."classifierId",v."diaObjectId",v."classId",v."alertId",v."trueClassId",v."probability",v."alertSentTimestamp", '
          '  s."midPointTai"-ot.peakmjd AS deltat '
          'FROM elasticc_view_sourceclassifications v '
          'INNER JOIN elasticc_diasource s ON v."diaSourceId"=s."diaSourceId" '
          'INNER JOIN elasticc_diaobjecttruth ot ON v."diaObjectId"=ot."diaObjectId" '
          'WHERE "classifierId" IN %(cfers)s AND "trueClassId"=111 '
          'ORDER BY "diaObjectId","alertSentTimestamp" '
          'LIMIT 10000' )
rows = run_query( query, { 'cfers': tuple(use_cfers.keys()) } )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

Query start at 2022-12-15T11:19:46.137525...
Query done at 2022-12-15T11:19:48.844291


,classifierId,diaObjectId,classId,alertId,trueClassId,probability,alertSentTimestamp,deltat
0,40,1000267,130,2000534000,111,0.000000,2022-12-04T22:34:16.368Z,-2.7723
1,40,1000267,120,2000534000,111,0.000000,2022-12-04T22:34:16.368Z,-2.7723
2,40,1000267,220,2000534000,111,0.000000,2022-12-04T22:34:16.368Z,-2.7723
3,40,1000267,215,2000534000,111,0.000000,2022-12-04T22:34:16.368Z,-2.7723
4,40,1000267,210,2000534000,111,0.000000,2022-12-04T22:34:16.368Z,-2.7723
...,...,...,...,...,...,...,...,...
9995,40,1008732,114,2017464032,111,0.000705,2022-10-07T02:41:38.848Z,0.4511
9996,40,1008732,113,2017464032,111,0.005752,2022-10-07T02:41:38.848Z,0.4511
9997,40,1008732,110,2017464032,111,0.005850,2022-10-07T02:41:38.848Z,0.4511
9998,40,1008732,132,2017464032,111,0.000019,2022-10-07T02:41:38.848Z,0.4511


In [23]:
data['diaObjectId'].unique()

array([1000267, 1000292, 1000316, 1000448, 1000674, 1000757, 1000776,
       1001197, 1001216, 1001372, 1001409, 1001747, 1002071, 1002288,
       1002647, 1002657, 1002842, 1002970, 1003312, 1003330, 1003377,
       1003402, 1003722, 1003806, 1003878, 1004142, 1004205, 1004280,
       1004461, 1004475, 1004497, 1004610, 1004688, 1004713, 1004893,
       1004913, 1004931, 1004978, 1005358, 1005627, 1005628, 1005646,
       1005738, 1005787, 1005839, 1005956, 1006052, 1006147, 1006335,
       1006434, 1006466, 1006592, 1006701, 1007021, 1007116, 1007230,
       1007364, 1007486, 1007857, 1007858, 1007987, 1008033, 1008462,
       1008473, 1008505, 1008579, 1008732])